In [ ]:
n = 6
init_tellers = 10 # A0
cost_salary = 10 #CS
cost_retire = 30 #CR
cost_hire = 10 #CH
retire_max = 10 #RM
max_labor = 160 #MH

hours_created = [4800 6000 6500 5000 4500 3500] #HrsReq (work created in each month)
max_hrs_left = [800 700 600 500 400 0]; #Lt (max hours left over at the end of each month)
